In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
from tensorflow.python.keras import layers, Sequential,losses, metrics, optimizers, callbacks, models
from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.applications import vgg16, resnet
from tensorflow.python.keras.optimizer_v2 import adam

In [2]:
image_height = 48
image_width = 48
emotions_count = 8
emotion_labels = ['neutral', 'happiness', 'surprise', 'sadness', 'anger', 'disgust', 'fear', 'contempt']

In [3]:
# dataset_index:
#     0: original samples
#     1: original samples + flip
#     2: original samples + flip ± rotation 10°
#     3: original samples + flip ± rotation 20°
#     4: original samples + flip ± rotation 30°
#     5: original samples + flip ± rotation 40°
# resize:
#     0: pixel value 0~255
#     1: pixel value -1~1
#     2: pixel value 0~1
def load_data(dataset_index, resize):
    if dataset_index == 0:
        image_path = "./dataset/images.npy"
        emotion_path = "./dataset/emotions_multi.npy"
        training_sample_times = 1
    if dataset_index == 1:
        image_path = "./dataset2/images.npy"
        emotion_path = "./dataset2/emotions_multi.npy"
        training_sample_times = 2
    if dataset_index == 2:
        image_path = "./dataset_f+r10/images.npy"
        emotion_path = "./dataset_f+r10/emotions_multi.npy"
        training_sample_times = 4
    if dataset_index == 3:
        image_path = "./dataset_f+r20/images.npy"
        emotion_path = "./dataset_f+r20/emotions_multi.npy"
        training_sample_times = 4
    if dataset_index == 4:
        image_path = "./dataset_f+r30/images.npy"
        emotion_path = "./dataset_f+r30/emotions_multi.npy"
        training_sample_times = 4
    if dataset_index == 5:
        image_path = "./dataset_f+r40/images.npy"
        emotion_path = "./dataset_f+r40/emotions_multi.npy"
        training_sample_times = 4
    images = np.load(image_path)
    images = tf.convert_to_tensor(images)
    if resize == 0:
        pass
    if resize == 1:
        images = layers.Rescaling(1./127.5, offset= -1)(images)
    if resize == 2:
        images = layers.Rescaling(1./255, offset= 0)(images)
    images = tf.image.grayscale_to_rgb(images)
    emotions = np.load(emotion_path)
    emotions = tf.convert_to_tensor(emotions)
    
    training_samples = 28317 * training_sample_times
    validation_samples = 3541 * training_sample_times
    training_size = training_samples + validation_samples
    
    training_images = images[:training_size]
    test_images = images[training_size:]
    training_emotions = emotions[:training_size]
    test_emotions = emotions[training_size:]
    return training_images, training_emotions, test_images, test_emotions

In [4]:
import gc
tf.config.run_functions_eagerly(True)
def model_acc(y_true, y_pred):
    size = y_true.shape[0]
    acc = 0
    for i in range(size):
        true = y_true[i]
        pred = y_pred[i]           
        index_max = tf.argmax(pred).numpy()
        if true[index_max].numpy()==tf.reduce_max(true).numpy():
            acc += 1
    return acc/size

In [5]:
def train(learning_rate, loss, num_epochs, batch_size, dataset_index, resize):
    model = create_model()
    model.compile(optimizer=adam.Adam(learning_rate=learning_rate), 
                  loss=loss, 
                  metrics = [model_acc])
    training_images, training_emotions, test_images, test_emotions = load_data(dataset_index, resize)
    history = model.fit(x=training_images,
                     y=training_emotions,
                     batch_size=batch_size,
                     epochs=num_epochs,
                     validation_data=(test_images, test_emotions))
    del model, training_images, training_emotions, test_images, test_emotions
    gc.collect()
    return history

In [6]:
def create_model():
    base_model = vgg16.VGG16(include_top=False, 
                             weights="imagenet", 
                             input_shape=(48,48,3))
    base_model.trainable=True
    return Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(4096, activation='relu'),
        layers.Dense(4096, activation='relu'),
        layers.Dense(emotions_count, activation='softmax'),
    ])

In [ ]:
if not os.path.isdir('./new-results/'):
    os.mkdir('./new-results/')

learning_rate = 1e-5
num_epochs = 40
loss = losses.MeanSquaredError()
batch_size = 32
dataset_index = 1
val_accs = []
for resize in range(3):
    history = train(learning_rate, loss, num_epochs, batch_size, dataset_index, resize)
    val_accs.append(history.history['val_model_acc'])
    if resize == 0:
        history_save_path = './new-results/0~255.txt'
    if resize == 1:
        history_save_path = './new-results/-1~1.txt'
    if resize == 2:
        history_save_path = './new-results/0~1.txt'
    with open(history_save_path, 'wb') as file_pi:
        pickle.dump(history.history, file_pi)

/userhome/2072/fyp21022/anaconda3/lib/python3.8/site-packages/tensorflow/python/data/ops/dataset_ops.py:4526: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/40
1992/1992 [==============================] - 226s 112ms/step - loss: 0.0324 - model_acc: 0.6552 - val_loss: 0.0256 - val_model_acc: 0.7166
Epoch 2/40
1992/1992 [==============================] - 226s 114ms/step - loss: 0.0203 - model_acc: 0.7652 - val_loss: 0.0226 - val_model_acc: 0.7425
Epoch 3/40
1376/1992 [===================>..........] - ETA: 1:07 - loss: 0.0159 - model_acc: 0.8070

In [ ]:
# model comparison
epochs = range(1, 1+num_epochs)
fig = plt.figure()
for resize in range(3):
    val_acc = val_accs[resize]
    if resize == 0:
        plot_label = '0~255'
    if resize == 1:
        plot_label = '-1~1'
    if resize == 2:
        plot_label = '0~1'
    plt.plot(epochs, val_acc, label = plot_label)
plt.legend()
plt.show()

# save plot
fig.savefig('./new-results/pixel value.png')